In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
target = "TotalGHGEmissions"

In [34]:
data_quanti[target].mean()

114.22142835504427

# Import des données

In [3]:
data_quanti = pd.read_csv('files_cleaned/data_quanti.csv')
data_quali = pd.read_csv('files_cleaned/data_quali.csv')
data_quanti = data_quanti.drop(columns=['Unnamed: 0'])

In [4]:
data_quanti = data_quanti.dropna(subset=[target])

In [5]:
x = data_quanti.drop(columns=["SiteEnergyUseWN(kBtu)", "TotalGHGEmissions",
                              "SiteEnergyUse(kBtu)",
                              'ENERGYSTARScore', 'GHGEmissionsIntensity', 
                              'Electricity(kWh)', 'Electricity(kBtu)',
                              'NaturalGas(kBtu)', 'NaturalGas(therms)',
                              'SiteEnergyUseWN(kBtu)', 'SiteEUIWN(kBtu/sf)',
                              'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
                              'SteamUse(kBtu)', 'OSEBuildingID', 'DataYear'])
y = data_quanti[[target]]
energy = data_quanti[['ENERGYSTARScore', 'GHGEmissionsIntensity', 
                      'Electricity(kWh)', 'Electricity(kBtu)',
                      'NaturalGas(kBtu)', 'NaturalGas(therms)',
                      'SiteEnergyUseWN(kBtu)', 'SiteEUIWN(kBtu/sf)',
                      'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
                      'SteamUse(kBtu)', "SiteEUI(kBtu/sf)"]]

In [6]:
x = x.fillna(x.mean())
y = y.astype(int)

# Division des datasets en deux 

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#On enlève du train les batiments ayant une trop grande consommation electrique
#z_train = pd.merge(y_train, x_train, left_index=True, right_index=True)
#z_train = z_train[z_train[target] < 50000000]
#x_train = z_train.drop(columns=[target])
#y_train = z_train[[target]]

On standardise les données 

In [9]:
from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)  

In [10]:
x_train.shape

(5281, 12)

# Mise en place d'une baseline

In [11]:
reg_simple = sm.OLS(y_train, x_train).fit()

In [12]:
from sklearn.metrics import mean_squared_error

In [13]:
y_predict = reg_simple.predict(x_test)

In [14]:
baseline_error = np.sqrt(mean_squared_error(y_test, y_predict))
baseline_error

561.7822394411616

# Modèles linéaires 

## Regression Ridge

In [15]:
n_alphas = 200
alphas = np.logspace(-5, 5, n_alphas)

In [16]:
from sklearn import linear_model
from sklearn.linear_model import Ridge
ridge = linear_model.Ridge()

coefs = []
errors = []
for a in alphas:
    ridge.set_params(alpha=a)
    ridge.fit(x_train, y_train)
    coefs.append(ridge.coef_)
    y_predict = ridge.predict(x_test)
    errors.append([baseline_error, 
                   np.sqrt(mean_squared_error(y_test, y_predict))])

In [17]:
min(errors)

[561.7822394411616, 390.8145830420948]

La regression Ridge est meilleure que la regression normale, cependant la mean squared error reste extrêmement élevée

## Regression Lasso

In [18]:
n_alphas = 600
alphas = np.logspace(-5, 5, n_alphas)
lasso = linear_model.Lasso(fit_intercept=False)

coefs = []
errors = []
for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(x_train, y_train)
    coefs.append(lasso.coef_)
    y_predict = lasso.predict(x_test)
    errors.append([baseline_error, 
                   np.sqrt(mean_squared_error(y_test, y_predict))])

C:\Users\jpaul\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\jpaul\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\jpaul\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\jpaul\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increa

C:\Users\jpaul\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\jpaul\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\jpaul\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\jpaul\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increa

In [19]:
min(errors)

[561.7822394411616, 392.06601621153857]

La régression Lasso est meilleure que la régression simple, cependant elle est moins bonne que la régression Ridge

## Regression ElasticNet

# Modèles non linéaires 

## KRR

Initialisation du modèle

In [20]:
from sklearn import kernel_ridge
predicteur = kernel_ridge.KernelRidge(
    alpha=1.0, # valeur par défaut 
    kernel='rbf', # noyau Gaussien
    gamma=0.01)   # valeur de 1/(2 * sigma**2)
                                     
# entraîner le classifieur sur le jeu d'entrainement
predicteur.fit(x_train, y_train)

# prédire sur le jeu de test
y_test_pred = predicteur.predict(x_test)

# calculer la RMSE sur le jeu de test
from sklearn import metrics
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print("RMSE: {:.2f}".format(rmse))

RMSE: 380.63


La MSE est ici inférieur à notre baseline et à la regression Ridge, le modèle est donc amélioré. Cependant cette dernière reste très élevée

Testons avec une boucle sur alpha :

In [21]:
n_alphas = 200
alphas = np.logspace(-5, 5, n_alphas)


errors = []

for a in alphas : 
    predicteur = kernel_ridge.KernelRidge(
        alpha=a, # valeur par défaut 
        kernel='rbf', # noyau Gaussien
        gamma=0.01)   # valeur de 1/(2 * sigma**2)
    
    # entraîner le classifieur sur le jeu d'entrainement
    predicteur.fit(x_train, y_train)

    # prédire sur le jeu de test
    y_predict = predicteur.predict(x_test)

    # calculer la MSE sur le jeu de test
    errors.append([baseline_error, 
                   np.sqrt(mean_squared_error(y_test, y_predict))])

In [22]:
min(errors)

[561.7822394411616, 351.0778716969234]

La MSE est ici bien plus basse, le modèle est donc bien plus performant ! 

MSE 3,2 millions

## Random Forest 

Préparation des données 

In [23]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

On commence avec une random forest sans cross-validation

In [24]:
from sklearn.ensemble import RandomForestRegressor

Initialisation du modèle avec 1000 arbres 

In [25]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(x_train, y_train)

C:\Users\jpaul\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

Prédictions et métrique d'erreur

In [26]:
# Use the forest's predict method on the test data
y_predict = rf.predict(x_test)

# CCalcul RMSE
errors = np.sqrt(mean_squared_error(y_test, y_predict))

# Print RMSE
print('Mean Squared Error:', errors)

Mean Squared Error: 213.85820252401453


Amélioration du modèle 

In [27]:
from sklearn.feature_selection import SelectFromModel

In [28]:
select = SelectFromModel(rf, prefit=True, threshold=0.003)
x_train2 = select.transform(x_train)
print(x_train2.shape)

(5281, 11)


Une feature a été enlevée

In [29]:
x_test2 = select.transform(x_test)

In [30]:
rf.fit(x_train2, y_train)
y_predict = rf.predict(x_test2)
errors = np.sqrt(mean_squared_error(y_test, y_predict))
print('Mean Squared Error:', errors)

C:\Users\jpaul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


Mean Squared Error: 213.02015917535925


Le modèle est amélioré, cependant la RMSE est toujours supérieur à notre KRR

In [31]:
y.median()

TotalGHGEmissions    33.0
dtype: float64

## Test Neural Network

In [32]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [33]:
"""import timeit

start_time = timeit.default_timer()

def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, x_train, y_train, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

elapsed = timeit.default_timer() - start_time

print("Time:", elapsed)"""

'import timeit\n\nstart_time = timeit.default_timer()\n\ndef larger_model():\n    # create model\n    model = Sequential()\n    model.add(Dense(12, input_dim=12, kernel_initializer=\'normal\', activation=\'relu\'))\n    model.add(Dense(6, kernel_initializer=\'normal\', activation=\'relu\'))\n    model.add(Dense(6, kernel_initializer=\'normal\', activation=\'relu\'))\n    model.add(Dense(1, kernel_initializer=\'normal\'))\n    # Compile model\n    model.compile(loss=\'mean_squared_error\', optimizer=\'adam\')\n    return model\n\n# evaluate model with standardized dataset\nestimators = []\nestimators.append((\'standardize\', StandardScaler()))\nestimators.append((\'mlp\', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))\npipeline = Pipeline(estimators)\nkfold = KFold(n_splits=10)\nresults = cross_val_score(pipeline, x_train, y_train, cv=kfold)\nprint("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))\n\nelapsed = timeit.default_timer() - start_time\n\

Point à revoir en détail (code juste copié et débuggué) <br/>
Source : 